In [1]:
import os

# spark_version
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudVineAnalysis").getOrCreate()

In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
kitchen_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
kitchen_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [4]:
vine_df = kitchen_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df = vine_df.drop_duplicates(["review_id"])
vine_df = vine_df.dropna()
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10002O5HSLMDW|          5|            0|          0|   N|
|R100042H9DN6G5|          5|            0|          0|   N|
|R100065LWA66A9|          4|            0|          1|   N|
|R10006YK5ARTDT|          5|            0|          0|   N|
|R100079NQX5FV0|          5|            0|          0|   N|
|R10009BPC6YR2T|          4|            0|          0|   N|
|R1000CEJZ30MZL|          5|            1|          2|   N|
|R1000CK3G7C8XX|          5|            0|          0|   N|
|R1000IX1RKMQ8T|          5|            0|          0|   N|
|R1000JEOH5IPCA|          4|            0|          0|   N|
|R1000L0JTNVOMK|          5|            0|          0|   N|
|R1000MJ0FG0INR|          4|            0|          0|   N|
|R1000OO0PP3MXV|          5|            0|          0|   N|
|R1000OU7YF4TL6|          5|            

In [30]:
# Number of total reviews
total_reviews = vine_df.count()
total_reviews

4880460

## Paid Reviews (part of Vine program)

In [5]:
# Reviews were written as part of the Vine program (people received some sort of payment for writing the reviews)
paid_reviews = vine_df.filter(vine_df['vine'] == "Y")
paid_reviews.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100J8EBA8YU2C|          5|            1|          2|   Y|
|R100MM5OJVG1ZJ|          5|            4|          5|   Y|
|R100MW0JY7FWC1|          1|            0|          0|   Y|
|R101T4MVDPSOFS|          5|            1|          1|   Y|
|R101YN74YBNGZ4|          4|            0|          0|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [6]:
# Summary Statistics 
paid_reviews.describe().show()

+-------+--------------+------------------+-----------------+-----------------+-----+
|summary|     review_id|       star_rating|    helpful_votes|      total_votes| vine|
+-------+--------------+------------------+-----------------+-----------------+-----+
|  count|         24434|             24434|            24434|            24434|24434|
|   mean|          null|4.2489154456904314|5.964557583694852|6.917492019317344| null|
| stddev|          null|0.8973440726829514|44.84597546541934|47.16703558474255| null|
|    min|R100C6GLMB9OWX|                 1|                0|                0|    Y|
|    max| RZZQVGFXIH3KD|                 5|             3325|             3428|    Y|
+-------+--------------+------------------+-----------------+-----------------+-----+



In [7]:
# Number of reviews as a part of Vine program 
paidrev_total = paid_reviews.count()
paidrev_total

24434

In [32]:
# Percentage of paid reviews
percent_paidrev = (paidrev_total / total_reviews) * 100
percent_paidrev

0.500649528937846

In [8]:
# Get all 5-star reviews (part of Vine)
FiveStar_paidrev_total = paid_reviews.filter(paid_reviews["star_rating"] == 5).count()
FiveStar_paidrev_total

11753

In [9]:
# Percentage of 5-star paid reviews
FiveStar_percent = (FiveStar_paidrev_total / paidrev_total) * 100
FiveStar_percent

48.1010067938119

In [10]:
# Get all 1-star reviews (part of Vine)
OneStar_paidrev_total = paid_reviews.filter(paid_reviews["star_rating"] == 1).count()
OneStar_paidrev_total


351

In [11]:
# Percentage of 1-star paid reviews
OneStar_percent = (OneStar_paidrev_total / paidrev_total) * 100
OneStar_percent

1.4365228779569452

In [12]:
from pyspark.sql.functions import desc

avg_ratings_vine =paid_reviews.select(["star_rating", "helpful_votes"])\
  .groupby("helpful_votes").agg({"star_rating": "avg", "helpful_votes":"count"})\
  .orderBy(desc("count(helpful_votes)"))

avg_ratings_vine.show(truncate=False)

+-------------+------------------+--------------------+
|helpful_votes|avg(star_rating)  |count(helpful_votes)|
+-------------+------------------+--------------------+
|0            |4.335774697009329 |11469               |
|1            |4.259237421383648 |5088                |
|2            |4.170140365801786 |2351                |
|3            |4.140253565768621 |1262                |
|4            |4.071979434447301 |778                 |
|5            |4.0980735551663745|571                 |
|6            |4.179878048780488 |328                 |
|7            |4.057142857142857 |280                 |
|8            |4.037383177570093 |214                 |
|9            |4.1454545454545455|165                 |
|10           |4.102362204724409 |127                 |
|12           |4.096             |125                 |
|13           |4.019047619047619 |105                 |
|11           |4.144230769230769 |104                 |
|14           |4.024390243902439 |82            

In [13]:
# Number of Helpfuls Votes with at least 1 vote
heplful_votes_paid = paid_reviews.filter(paid_reviews['helpful_votes'] >= 1).count()
heplful_votes_paid 

12965

In [28]:
# Percentage of Helpfuls Votes with at least 1 vote
heplful_percent_paidv = (heplful_votes_paid / paidrev_total) * 100
heplful_percent_paidv

53.06130801342391

## Unpaid Reviews (not part of Vine program)

In [15]:
# Reviews were writen not related to Vine Program (these people haven't received payment or free products for writing reviews)
unpaid_reviews = vine_df.filter(vine_df['vine'] == "N")
unpaid_reviews.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10002O5HSLMDW|          5|            0|          0|   N|
|R100042H9DN6G5|          5|            0|          0|   N|
|R100065LWA66A9|          4|            0|          1|   N|
|R10006YK5ARTDT|          5|            0|          0|   N|
|R100079NQX5FV0|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [16]:
# Summary Statistics 
unpaid_reviews.describe().show()

+-------+--------------+-----------------+------------------+------------------+-------+
|summary|     review_id|      star_rating|     helpful_votes|       total_votes|   vine|
+-------+--------------+-----------------+------------------+------------------+-------+
|  count|       4856026|          4856026|           4856026|           4856026|4856026|
|   mean|          null|4.207215323805927| 2.227274730407127|2.6559099971870004|   null|
| stddev|          null|1.288595147626843|22.752191578607952|23.919659731444806|   null|
|    min|R10002O5HSLMDW|                1|                 0|                 0|      N|
|    max| RZZZZPTRHB6UP|                5|             11173|             11501|      N|
+-------+--------------+-----------------+------------------+------------------+-------+



In [17]:
# Number of reviews which aren't part of Vine program
unpaidrev_total = unpaid_reviews.count()
unpaidrev_total

4856026

In [31]:
# Percentage of paid reviews
percent_unpaidrev = (unpaidrev_total / total_reviews) * 100
percent_unpaidrev

99.49935047106216

In [18]:
# Get all 5-star reviews (not part of Vine)
FiveStar_unpaidrev_total = unpaid_reviews.filter(unpaid_reviews["star_rating"] == 5).count()
FiveStar_unpaidrev_total

3116807

In [19]:
# Percentage of 5-star unpaid reviews
FiveStar_nonvine_percent = (FiveStar_unpaidrev_total / unpaidrev_total) * 100
FiveStar_nonvine_percent

64.18431449913983

In [20]:
# Get all 1-star reviews (part of Vine)
OneStar_unpaidrev_total = unpaid_reviews.filter(unpaid_reviews["star_rating"] == 1).count()
OneStar_unpaidrev_total

426954

In [21]:
# Percentage of 1-star paid reviews
OneStar_nonvine_percent = (OneStar_unpaidrev_total / unpaidrev_total) * 100
OneStar_nonvine_percent

8.7922511123293

In [22]:
avg_ratings_nonvine = unpaid_reviews.select(["star_rating", "helpful_votes"])\
  .groupby("helpful_votes").agg({"star_rating": "avg", "helpful_votes":"count"})\
  .orderBy(desc("count(helpful_votes)"))

avg_ratings_nonvine.show(truncate=False)

+-------------+------------------+--------------------+
|helpful_votes|avg(star_rating)  |count(helpful_votes)|
+-------------+------------------+--------------------+
|0            |4.38601250538644  |3165356             |
|1            |4.084310069786682 |757881              |
|2            |3.84449786528915  |287861              |
|3            |3.730583054068282 |155766              |
|4            |3.6629369316759823|95896               |
|5            |3.6154126103175326|65946               |
|6            |3.608359261563991 |48210               |
|7            |3.5892224449998626|36409               |
|8            |3.555955040491483 |28648               |
|9            |3.5617567919451436|23042               |
|10           |3.569372693726937 |18970               |
|11           |3.587136274695449 |15843               |
|12           |3.5567955066144408|13531               |
|13           |3.5443669851105946|11619               |
|14           |3.572567940133911 |10156         

In [23]:
# Number of Helpfuls Votes with at least 1 vote
heplful_votes_unpaid = unpaid_reviews.filter(unpaid_reviews['helpful_votes'] >= 1).count()
heplful_votes_unpaid 

1690670

In [29]:
# Percentage of Helpfuls Votes with at least 1 vote
heplful_percent_unpaidv = (heplful_votes_unpaid / unpaidrev_total) * 100
heplful_percent_unpaidv

34.81591737770762